In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_circuits
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/circuits.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_circuits (
  CONSTRAINT valid_id EXPECT (circuitId IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_name EXPECT (name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_location EXPECT (location IS NOT NULL)
)
COMMENT 'Cleaned circuits data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT circuitId,
       name,
       location
  FROM STREAM(bronze_circuits)